<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#Sample-sheets" data-toc-modified-id="Sample-sheets-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Sample sheets</a></span></li><li><span><a href="#Merged-post-QC-sequences-from-all-existing-runs" data-toc-modified-id="Merged-post-QC-sequences-from-all-existing-runs-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Merged post-QC sequences from all existing runs</a></span></li><li><span><a href="#Sample-metadata" data-toc-modified-id="Sample-metadata-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Sample metadata</a></span></li></ul></li><li><span><a href="#Low-coverage-samples" data-toc-modified-id="Low-coverage-samples-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Low coverage samples</a></span></li><li><span><a href="#Selecting-libs-from-prev-runs" data-toc-modified-id="Selecting-libs-from-prev-runs-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Selecting libs from prev runs</a></span><ul class="toc-item"><li><span><a href="#Summary-of-repool" data-toc-modified-id="Summary-of-repool-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Summary of repool</a></span></li><li><span><a href="#Overlapping-barcodes?" data-toc-modified-id="Overlapping-barcodes?-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Overlapping barcodes?</a></span></li></ul></li><li><span><a href="#Creating-pooling-table" data-toc-modified-id="Creating-pooling-table-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Creating pooling table</a></span><ul class="toc-item"><li><span><a href="#Repool:-round-1" data-toc-modified-id="Repool:-round-1-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Repool: round 1</a></span><ul class="toc-item"><li><span><a href="#Writing-table" data-toc-modified-id="Writing-table-7.1.1"><span class="toc-item-num">7.1.1&nbsp;&nbsp;</span>Writing table</a></span></li></ul></li><li><span><a href="#Repool:-round2" data-toc-modified-id="Repool:-round2-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Repool: round2</a></span><ul class="toc-item"><li><span><a href="#Writing-tabel" data-toc-modified-id="Writing-tabel-7.2.1"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span>Writing tabel</a></span></li></ul></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Determine which samples to re-pool & re-sequence due to low coverage

# Var

In [4]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGQC/'

# sample sheets for each run
run0126_file = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRun0126/SampleSheet_HiSeq_R0126_L2.tsv'
run0133_redo_file = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRun0133/SampleSheet_HiSeq_R0133_L3_corrected.tsv'
run0133_repool_file = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRun0133/SampleSheet_HiSeq_R0133_L4_corrected.tsv'
run0138_file = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRun0138/SampleSheet_HiSeq_Run138_L8_Georg.tsv'
run0163_file = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRun0163/SampleSheet_HiSeq_Run163_L3.tsv'

# post-QC sequence summary
qc_file = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGQC/sample_qc_summary.tsv'

# sample metadata
metadata_file = '/ebio/abt3_projects/Georg_animal_feces/data/mapping/unified_metadata_complete_190529.tsv'


# Init

In [5]:
library(dplyr)
library(tidyr)
library(ggplot2)
source('/ebio/abt3_projects/Georg_animal_feces/code/misc_r_functions/init.R')

In [6]:
dt.dims()

In [87]:
#' Output: named vector (Well --> location); column-wise location
well96_index = function(){
  wells = c('A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1',
            'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2',
            'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3',
            'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4',
            'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5',
            'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6',
            'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7',
            'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8',
            'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9',
            'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10',
            'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11',
            'A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12')
  idx = 1:length(wells)
  names(idx) = wells
  return(idx)
}

In [93]:
#' input: character vector of wellIDs
well2index = function(x, plate_type='96-well'){
  x = gsub('([A-Z])0([0-9])', '\\1\\2', as.character(x))
  if(plate_type == '96-well'){
    idx = well96_index()
  } else
  if(plate_type == '384-well'){
    idx = well384_index()
  } else {
    stop('Do not recoginize plate type')
  }
  return(as.numeric(idx[x]))
}

# Load

## Sample sheets

In [19]:
files = c(run0126_file, run0133_redo_file, run0133_repool_file, run0138_file, run0163_file)
read_sheet = function(F){
    df = read.delim(F, sep='\t')
    df$run = gsub('HiSeqRun', '', basename(dirname(F)))
    return(df)
}
samp_sheets = files %>% 
    lapply(read_sheet) %>%
    do.call(rbind, .) %>%
    mutate(run = run %>% as.numeric,
           Sample_ID = gsub('^(F[0-9])', 'X\\1', Sample_ID))

samp_sheets

Lane,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description,run
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<fct>,<fct>,<fct>,<dbl>
1,X1_Eurasian_Beaver,NA,NA,NA,1,TAAGGCGA,1,GCGATCTA,Georg_animal_feces,nyoungblut,126
1,X2_Raccoon_Dog,NA,NA,NA,2,CGTACTAG,1,GCGATCTA,Georg_animal_feces,nyoungblut,126
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
3,X434_Blue_Wildebeest,X434_Blue_Wildebeest,plate2,E01,N716,ACTCGCTA,S519,ATGTCAAT,Georg_animal_gut_metagenome,sdauser,163
3,X435_Bactrian_Camel,X435_Bactrian_Camel,plate2,F01,N718,GGAGCTAC,S519,ATGTCAAT,Georg_animal_gut_metagenome,sdauser,163


## Merged post-QC sequences from all existing runs

In [20]:
qc = read.delim(qc_file, sep='\t')
qc

Sample,Total.Sequences,nonpareil_coverage,nonpareil_diversity
<fct>,<int>,<dbl>,<dbl>
X101_Horse,7716507,0.3414071,21.56971
X107_House_Mouse,2833353,0.4244829,19.91179
⋮,⋮,⋮,⋮
X91_Dunnock,3881823,0.5144225,20.01099
X92_Lesser_White_toothed_Shrew,1253488,0.4813252,19.16230


## Sample metadata

In [21]:
metadata = read.delim(metadata_file, sep='\t')  %>%
    dplyr::select(SampleID, scientific_name, class, order, family, genus,
                  habitat, diet, sample_type) %>%
    mutate(SampleID = gsub('^F', 'XF', SampleID))
metadata

SampleID,scientific_name,class,order,family,genus,habitat,diet,sample_type
<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
XF14_Common_Bream,Abramis_brama,Actinopterygii,Cypriniformes,Cyprinidae,Abramis,aquatic habitat,omnivore,feces
XF35_Red_Deer,Cervus_elaphus,Mammalia,Artiodactyla,Cervidae,Cervus,terrestrial habitat,herbivore,feces
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X189_fail_Rainbow_Trout,Oncorhynchus_mykiss,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,aquatic habitat,carnivore,feces
X232_fail_Brown_Trout,Salmo_trutta,Actinopterygii,Salmoniformes,Salmonidae,Salmo,aquatic habitat,carnivore,gut


# Low coverage samples

In [23]:
qc_f = qc %>%
    filter(nonpareil_coverage < 0.5)

qc_f

Sample,Total.Sequences,nonpareil_coverage,nonpareil_diversity
<fct>,<int>,<dbl>,<dbl>
X101_Horse,7716507,0.3414071,21.56971
X107_House_Mouse,2833353,0.4244829,19.91179
⋮,⋮,⋮,⋮
X82_Eurasian_Ruffe,1273012,0.3943804,19.57361
X92_Lesser_White_toothed_Shrew,1253488,0.4813252,19.16230


In [28]:
qc_f_j = qc_f %>%
    inner_join(samp_sheets, c('Sample'='Sample_ID')) 
qc_f_j$Sample %>% unique %>% length %>% print
qc_f_j

Warning message:
“Column `Sample`/`Sample_ID` joining factor and character vector, coercing into character vector”


[1] 132


Sample,Total.Sequences,nonpareil_coverage,nonpareil_diversity,Lane,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description,run
<chr>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<fct>,<fct>,<fct>,<dbl>
X101_Horse,7716507,0.3414071,21.56971,1,NA,NA,NA,7,CTCTCTAC,11,CGTAAGGA,Georg_animal_feces,nyoungblut,126
X101_Horse,7716507,0.3414071,21.56971,4,NA,1,G11,N707,CTCTCTAC,S511,CGTAAGGA,Georg_animal_feces,sdauser,133
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X92_Lesser_White_toothed_Shrew,1253488,0.4813252,19.1623,3,NA,plate1,A11,N706,TAGGCATG,S502,ATAGAGAG,Georg_animal_feces,sdauser,133
X92_Lesser_White_toothed_Shrew,1253488,0.4813252,19.1623,8,X92_Lesser_White_toothed_Shrew,plate1,A11,N706,TAGGCATG,S502,ATAGAGAG,Georg_animal_feces,sdauser,138


# Selecting libs from prev runs

In [106]:
qc_f_j %>%
    group_by(run) %>%
    summarize(n_samps = n_distinct(Sample)) %>%
    ungroup()

run,n_samps
<dbl>,<int>
126,120
133,131
138,116
163,80


In [116]:
# most recent
qc_f_j_R163 = qc_f_j %>%
    filter(run == 163) %>%
    mutate(Sample_Plate = gsub('plate', '', Sample_Plate))

qc_f_j_R163$nonpareil_coverage %>% summary
qc_f_j_R163

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.1507  0.3506  0.4202  0.4007  0.4676  0.4997 

Sample,Total.Sequences,nonpareil_coverage,nonpareil_diversity,Lane,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description,run
<chr>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<fct>,<fct>,<fct>,<dbl>
X10_House_Mouse,4643882,0.4148921,21.25512,3,X10_House_Mouse,1,C05,N701,TAAGGCGA,S502,ATAGAGAG,Georg_animal_gut_metagenome,sdauser,163
X110_Noctule,4775749,0.4049221,21.51596,3,X110_Noctule,1,F01,N708,CAGAGAGG,S512,CCACGCGT,Georg_animal_gut_metagenome,sdauser,163
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X365_fail_Goldmans_Ichthyomyine,1801301,0.4832537,19.55062,3,X365_fail_Goldmans_Ichthyomyine,1,G08,N705,GGACTCCT,S511,CGTAAGGA,Georg_animal_gut_metagenome,sdauser,163
X393_European_Rabbit,436561,0.3996944,18.33490,3,X393_European_Rabbit,1,H07,N702,CGTACTAG,S513,GGAGTTCC,Georg_animal_gut_metagenome,sdauser,163


In [117]:
# next most recent
qc_f_j_R138 = qc_f_j %>%
    anti_join(qc_f_j_R163, c('Sample')) %>%
    filter(run == 138) %>%
    mutate(Sample_Plate = gsub('plate', '', Sample_Plate))

qc_f_j_R138$nonpareil_coverage %>% summary
qc_f_j_R138

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2874  0.3663  0.4315  0.4141  0.4729  0.4969 

Sample,Total.Sequences,nonpareil_coverage,nonpareil_diversity,Lane,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description,run
<chr>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<fct>,<fct>,<fct>,<dbl>
X107_House_Mouse,2833353,0.4244829,19.91179,8,X107_House_Mouse,1,E12,N705,GGACTCCT,S512,CCACGCGT,Georg_animal_feces,sdauser,138
X115_Common_Kestrel,5513971,0.4969480,20.47736,8,X115_Common_Kestrel,2,D1,N704,TCCTGAGC,S513,GGAGTTCC,Georg_animal_feces,sdauser,138
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X82_Eurasian_Ruffe,1273012,0.3943804,19.57361,8,X82_Eurasian_Ruffe,1,H9,N704,TCCTGAGC,S502,ATAGAGAG,Georg_animal_feces,sdauser,138
X92_Lesser_White_toothed_Shrew,1253488,0.4813252,19.16230,8,X92_Lesser_White_toothed_Shrew,1,A11,N706,TAGGCATG,S502,ATAGAGAG,Georg_animal_feces,sdauser,138


In [118]:
### next most recent
qc_f_j_R133L3 = qc_f_j %>%
    anti_join(qc_f_j_R163, c('Sample')) %>%
    anti_join(qc_f_j_R138, c('Sample')) %>%
    filter(run == 133, Lane == 3)%>%
    mutate(Sample_Plate = gsub('plate', '', Sample_Plate))

qc_f_j_R133L3$nonpareil_coverage %>% summary
qc_f_j_R133L3

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2006  0.2601  0.3180  0.3276  0.3855  0.4739 

Sample,Total.Sequences,nonpareil_coverage,nonpareil_diversity,Lane,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description,run
<chr>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<fct>,<fct>,<fct>,<dbl>
X313_Red_Deer,7175977,0.2798983,21.83558,3,NA,3,H6,N708,CAGAGAGG,S509,GAGTAGCC,Georg_animal_feces,sdauser,133
X349_Smooth_Newt,9758382,0.2005695,22.56520,3,NA,3,B10,N708,CAGAGAGG,S510,GTCTGAGG,Georg_animal_feces,sdauser,133
X424_Quagga,7055376,0.3560308,21.49890,3,NA,4,G6,N708,CAGAGAGG,S517B,TGGCGGTC,Georg_animal_feces,sdauser,133
X430_Giraffe,11095316,0.4738784,21.14643,3,NA,5,F5,N704,TCCTGAGC,S519,ATGTCAAT,Georg_animal_feces,sdauser,133


In [119]:
### next most recent
qc_f_j_R133L4 = qc_f_j %>%
    anti_join(qc_f_j_R163, c('Sample')) %>%
    anti_join(qc_f_j_R138, c('Sample')) %>%
    anti_join(qc_f_j_R133L3, c('Sample')) %>%
    filter(run == 133, Lane == 4) %>%
    mutate(Sample_Plate = gsub('plate', '', Sample_Plate))

qc_f_j_R133L4$nonpareil_coverage %>% summary
qc_f_j_R133L4

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.2776  0.3244  0.4008  0.3942  0.4750  0.4825 

Sample,Total.Sequences,nonpareil_coverage,nonpareil_diversity,Lane,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description,run
<chr>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<fct>,<fct>,<fct>,<dbl>
X101_Horse,7716507,0.3414071,21.56971,4,NA,1,G11,N707,CTCTCTAC,S511,CGTAAGGA,Georg_animal_feces,sdauser,133
X108_Striped_Desert_Hamster,4501530,0.4824608,20.27906,4,NA,1,F12,N706,TAGGCATG,S512,CCACGCGT,Georg_animal_feces,sdauser,133
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X146_Cattle,7535123,0.2776102,21.86113,4,NA,2,H4,N708,CAGAGAGG,S516,TAACCGCG,Georg_animal_feces,sdauser,133
X98_Eurasian_Red_Squirrel,9259271,0.4763292,21.16895,4,NA,1,F11,N706,TAGGCATG,S511,CGTAAGGA,Georg_animal_feces,sdauser,133


In [122]:
# all combined
qc_repool = qc_f_j_R163 %>%
    rbind(qc_f_j_R138) %>%
    rbind(qc_f_j_R133L3) %>%
    rbind(qc_f_j_R133L4) %>%
    unite(run_lane_plate, run, Lane, Sample_Plate, remove=FALSE, sep='_')

qc_repool

Sample,Total.Sequences,nonpareil_coverage,nonpareil_diversity,run_lane_plate,Lane,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description,run
<chr>,<int>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<fct>,<fct>,<fct>,<dbl>
X10_House_Mouse,4643882,0.4148921,21.25512,163_3_1,3,X10_House_Mouse,1,C05,N701,TAAGGCGA,S502,ATAGAGAG,Georg_animal_gut_metagenome,sdauser,163
X110_Noctule,4775749,0.4049221,21.51596,163_3_1,3,X110_Noctule,1,F01,N708,CAGAGAGG,S512,CCACGCGT,Georg_animal_gut_metagenome,sdauser,163
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X146_Cattle,7535123,0.2776102,21.86113,133_4_2,4,NA,2,H4,N708,CAGAGAGG,S516,TAACCGCG,Georg_animal_feces,sdauser,133
X98_Eurasian_Red_Squirrel,9259271,0.4763292,21.16895,133_4_1,4,NA,1,F11,N706,TAGGCATG,S511,CGTAAGGA,Georg_animal_feces,sdauser,133


## Summary of repool

In [127]:
qc_repool$Total.Sequences %>% summary
qc_repool$nonpareil_coverage %>% summary

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
  118337  1223102  2620123  3217894  4511574 11095316 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.1507  0.3499  0.4223  0.4022  0.4709  0.4997 

In [130]:
# total number of plates
qc_repool$run_lane_plate %>% table
qc_repool$run_lane_plate %>% unique %>% length

.
133_3_3 133_3_4 133_3_5 133_4_1 133_4_2 138_8_1 138_8_2 138_8_3 138_8_4 163_3_1 
      2       1       1       3       4      14      10      10       6      74 
163_3_2 
      6 

[1] 11

In [131]:
qc_repool_j = qc_repool %>%
    inner_join(metadata, c('Sample'='SampleID')) 

dt.dims(10)
qc_repool_j %>%
    group_by(class) %>%
    summarize(n_samps = n_distinct(Sample)) %>%
    ungroup() %>%
    arrange(-n_samps)
dt.dims()

class,n_samps
<fct>,<int>
Mammalia,93
Aves,19
Actinopterygii,9
Reptilia,6
Amphibia,4


## Overlapping barcodes?

In [132]:
qc_repool %>%
    group_by(I7_Index_ID, I5_Index_ID) %>%
    summarize(n = n()) %>%
    ungroup() %>%
    filter(n > 1)

I7_Index_ID,I5_Index_ID,n
<chr>,<chr>,<int>


In [170]:
# duplicate samples?
qc_repool$Sample %>% length
qc_repool$Sample %>% unique %>% length

[1] 131

[1] 131

# Creating pooling table

* for robot pooling app

## Repool: round 1

In [163]:
source_plates = c('133_3_3', '133_3_4', '133_3_5', '133_4_1', '133_4_2', '163_3_1', '163_3_2')

In [164]:
plate_base = '96 Well Eppendorf TwinTec PCR[00'

qc_repool_f = qc_repool %>%
    filter(run_lane_plate %in% source_plates) %>%
    mutate(plate = run_lane_plate %>% as.factor %>% as.numeric) %>%
    dplyr::select(Sample, Sample_Plate, Sample_Well, I7_Index_ID, index,
                  I5_Index_ID, index2, run, Lane, plate) %>%
    mutate(`Source Labware Label` = gsub('([0-9]+)', '96 Well Eppendorf TwinTec PCR[00\\1]', plate)) %>%
    mutate(`Source Position` = well2index(Sample_Well)) %>%
    mutate(`Destination Labware Label` = '96 Well Eppendorf TwinTec PCR[005]') %>%
    arrange(`Source Labware Label`, `Source Position`) %>%
    mutate(`Destination Position` = 1:nrow(.),
           Volume = 5) %>%
    dplyr::select(`Source Labware Label`, `Source Position`, `Destination Labware Label`,
                  `Destination Position`, Volume, 
                  Sample, I7_Index_ID, index, I5_Index_ID, index2, run, Lane) 


qc_repool_f

Source Labware Label,Source Position,Destination Labware Label,Destination Position,Volume,Sample,I7_Index_ID,index,I5_Index_ID,index2,run,Lane
<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<fct>,<chr>,<fct>,<dbl>,<int>
96 Well Eppendorf TwinTec PCR[001],48,96 Well Eppendorf TwinTec PCR[005],1,5,X313_Red_Deer,N708,CAGAGAGG,S509,GAGTAGCC,133,3
96 Well Eppendorf TwinTec PCR[001],74,96 Well Eppendorf TwinTec PCR[005],2,5,X349_Smooth_Newt,N708,CAGAGAGG,S510,GTCTGAGG,133,3
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
96 Well Eppendorf TwinTec PCR[007],5,96 Well Eppendorf TwinTec PCR[005],90,5,X434_Blue_Wildebeest,N716,ACTCGCTA,S519,ATGTCAAT,163,3
96 Well Eppendorf TwinTec PCR[007],6,96 Well Eppendorf TwinTec PCR[005],91,5,X435_Bactrian_Camel,N718,GGAGCTAC,S519,ATGTCAAT,163,3


### Writing table

In [165]:
outfile = file.path(work_dir, '2019-01-22_to-repool_round1.tsv') 
write_table(qc_repool_f, outfile)

File written: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGQC//2019-01-22_to-repool_round1.tsv

## Repool: round2

In [166]:
source_plates = c('138_8_1', '138_8_2', '138_8_3', '138_8_4')

In [167]:
plate_base = '96 Well Eppendorf TwinTec PCR[00'

qc_repool_f = qc_repool %>%
    filter(run_lane_plate %in% source_plates) %>%
    mutate(plate = run_lane_plate %>% as.factor %>% as.numeric) %>%
    dplyr::select(Sample, Sample_Plate, Sample_Well, I7_Index_ID, index,
                  I5_Index_ID, index2, run, Lane, plate) %>%
    mutate(`Source Labware Label` = gsub('([0-9]+)', '96 Well Eppendorf TwinTec PCR[00\\1]', plate)) %>%
    mutate(`Source Position` = well2index(Sample_Well)) %>%
    mutate(`Destination Labware Label` = '96 Well Eppendorf TwinTec PCR[005]') %>%
    arrange(`Source Labware Label`, `Source Position`) %>%
    mutate(`Destination Position` = 1:nrow(.),
           Volume = 5) %>%
    dplyr::select(`Source Labware Label`, `Source Position`, `Destination Labware Label`,
                  `Destination Position`, Volume, 
                  Sample, I7_Index_ID, index, I5_Index_ID, index2, run, Lane) 

qc_repool_f

Source Labware Label,Source Position,Destination Labware Label,Destination Position,Volume,Sample,I7_Index_ID,index,I5_Index_ID,index2,run,Lane
<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<chr>,<fct>,<chr>,<fct>,<dbl>,<int>
96 Well Eppendorf TwinTec PCR[001],6,96 Well Eppendorf TwinTec PCR[005],1,5,X6_Common_Hamster,N706,TAGGCATG,S501,GCGATCTA,138,8
96 Well Eppendorf TwinTec PCR[001],8,96 Well Eppendorf TwinTec PCR[005],2,5,X8_Garden_Dormouse,N708,CAGAGAGG,S501,GCGATCTA,138,8
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
96 Well Eppendorf TwinTec PCR[004],61,96 Well Eppendorf TwinTec PCR[005],39,5,XF144_Wood_Sandpiper,N715,ATCTCAGG,S520,AGTTGGCT,138,8
96 Well Eppendorf TwinTec PCR[004],67,96 Well Eppendorf TwinTec PCR[005],40,5,XF315_Northern_Pike,N711,AAGAGGCA,S521,ACCTAGTA,138,8


### Writing tabel

In [168]:
outfile = file.path(work_dir, '2019-01-22_to-repool_round2.tsv') 
write_table(qc_repool_f, outfile)

File written: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGQC//2019-01-22_to-repool_round2.tsv

# sessionInfo

In [76]:
 sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/tidyverse/lib/libopenblasp-r0.3.7.so

Random number generation:
 RNG:     Mersenne-Twister 
 Normal:  Inversion 
 Sample:  Rounding 
 
locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] ggplot2_3.2.1 tidyr_1.0.0   dplyr_0.8.3  

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.3       magrittr_1.5     munsell_0.5.0    tidyselect_0.2.5
 [5] uuid_0.1-2       